# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [14]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("data/soil_measures.csv")

In [3]:
print(crops.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None


In [4]:
print(crops['crop'].unique())

['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


In [5]:
crops['crop'].value_counts()

crop
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

In an inintial inspection of the dataset, we found that there is no missing value in the dataframe, which is ideal for building machine learning models. 

Nitrogen, Phosphorous, Potassium and pH value are correctly shown as number type. 

22 different types of crops are recorded in the dataset, and the class are very balanced (each type of the crops all have 100 data points)

We will do an exploratory data analysis to check the quality of data and try to have an understanding of the whole dataset.

In [6]:
import plotly.express as px

nutritions = crops.drop('crop', axis=1)
nutritions = nutritions.melt(var_name='Variable', value_name='Value')
fig = px.box(nutritions, x='Variable', y='Value')
fig.show()

In [7]:
ph = nutritions[nutritions['Variable'] == 'ph']
fig_ph = px.box(ph, x='Variable', y='Value')
fig_ph.show()

K has some outliers. Ph in this dataset is distributed reasonably, between 3 to 10.

For the outliers in K. We would like to see if it is related to some certian crops.

In [8]:
high_k = crops[crops['K'] > 190]
high_k['crop'].value_counts()

crop
grapes    100
apple     100
Name: count, dtype: int64

This datasets contain some outliers in Potassium content level that is greater than 190. We found that the soil with extremely high level of potassium seems to be an ideal condition for growing grapes and apple.

## Prediction Model for the Crops Types

Before building a model, we need to do some transformation on the dataset. We need to transform the crops column into numeric, as machine learning models do not surpport non-numeric data.

In [9]:
# Transform the target variable
crops['crop_code'], unique = pd.factorize(crops['crop'])

# Here is the dictionary of the label and its corresponding crop
print(dict(enumerate(unique)))

{0: 'rice', 1: 'maize', 2: 'chickpea', 3: 'kidneybeans', 4: 'pigeonpeas', 5: 'mothbeans', 6: 'mungbean', 7: 'blackgram', 8: 'lentil', 9: 'pomegranate', 10: 'banana', 11: 'mango', 12: 'grapes', 13: 'watermelon', 14: 'muskmelon', 15: 'apple', 16: 'orange', 17: 'papaya', 18: 'coconut', 19: 'cotton', 20: 'jute', 21: 'coffee'}


Then we start to build the model

In [10]:
#Split the data into train and test set
X = crops.drop(['crop', 'crop_code'], axis=1)
y = crops['crop_code']

print(X.shape, y.shape)

(2200, 4) (2200,)


In [11]:
feature_dict = {}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)
for feature in ['N', 'P', 'K', 'ph']:
    log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    log_reg.fit(X_train[[feature]], y_train)  # Reshape to 2D array
    y_pred = log_reg.predict(X_test[[feature]])  # Reshape to 2D array
    feature_performance = metrics.f1_score(y_test, y_pred, average='weighted')
    print(f"F1-score for {feature}: {feature_performance}")
    feature_dict[feature] = feature_performance

F1-score for N: 0.09628928316058114
F1-score for P: 0.1167562881317604
F1-score for K: 0.2222911655508141
F1-score for ph: 0.03488363219716933


/Users/yrwang/Desktop/datafun/Datacamp_projects_portfolio/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.

/Users/yrwang/Desktop/datafun/Datacamp_projects_portfolio/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning:

lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/Users/yrwang/Desktop/datafun/Datacamp_projects_portfolio/.venv/lib/python

In [12]:
feature_dict

{'N': 0.09628928316058114,
 'P': 0.1167562881317604,
 'K': 0.2222911655508141,
 'ph': 0.03488363219716933}

What is the best predictive feature for the crop?

In [13]:
best_predictive_feature = {'K': feature_dict['K']}
print(best_predictive_feature)

{'K': 0.2222911655508141}
